In [2]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 데이터 불러오기
군집화에 사용한 몇 개의 변수들만 가진 데이터 사용.

In [3]:
df = pd.read_csv('data/pdde_offline_all_4.csv', encoding='utf-8')
df.head()

,고객번호,성별,연령대,거주지분류대코드,영수증번호,구매일자,채널구분,제휴사,상품코드,상품대분류명,상품중분류명,상품소분류명,구매시간,구매금액,구매수량,cluster,구매월
0,M961815188,여성,40대,Z10,A02148492505,20210801,1,A02,PD1508,축산물,국산소고기,한우앞다리,16,19560,1,0,08월
1,M934362391,여성,40대,Z10,A02197599530,20211214,1,A02,PD1160,유제품,유가공품,치즈,22,4980,1,2,12월
2,M184315408,여성,30대,Z11,A01335067549,20210906,1,A01,PD0777,스포츠패션,남성스포츠화,남성런닝/트레이닝화,16,124000,1,1,09월
3,M805930072,여성,40대,Z13,A02348434840,20211226,1,A02,PD1014,완구,교육완구,레고,15,149900,1,2,12월
4,M590177102,여성,50대,Z10,A02092460645,20210608,1,A02,PD1173,음료,생수,생수,20,5880,1,1,06월


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 17 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   고객번호      300000 non-null  object
 1   성별        300000 non-null  object
 2   연령대       300000 non-null  object
 3   거주지분류대코드  300000 non-null  object
 4   영수증번호     300000 non-null  object
 5   구매일자      300000 non-null  int64 
 6   채널구분      300000 non-null  int64 
 7   제휴사       300000 non-null  object
 8   상품코드      300000 non-null  object
 9   상품대분류명    300000 non-null  object
 10  상품중분류명    300000 non-null  object
 11  상품소분류명    300000 non-null  object
 12  구매시간      300000 non-null  int64 
 13  구매금액      300000 non-null  int64 
 14  구매수량      300000 non-null  int64 
 15  cluster   300000 non-null  int64 
 16  구매월       300000 non-null  object
dtypes: int64(6), object(11)
memory usage: 38.9+ MB


## 군집화에 사용한 변수 선택

In [9]:
df = df[['성별', '연령대', '거주지분류대코드', '제휴사', '상품대분류명', '구매시간', '구매금액', '구매수량', '구매월', 'cluster']]
df.head()

,성별,연령대,거주지분류대코드,제휴사,상품대분류명,구매시간,구매금액,구매수량,구매월,cluster
0,여성,40대,Z10,A02,축산물,16,19560,1,08월,0
1,여성,40대,Z10,A02,유제품,22,4980,1,12월,2
2,여성,30대,Z11,A01,스포츠패션,16,124000,1,09월,1
3,여성,40대,Z13,A02,완구,15,149900,1,12월,2
4,여성,50대,Z10,A02,음료,20,5880,1,06월,1


결측치는 없음을 알 수 있다.

## 범주형 변수 수치화
주로 오디널인코딩 사용해서 수치화. 트리 계열 모델은 오디널인코딩에서 순위 영향을 받지 않는다고 한다.

1. 성별  
명목변수 - 원핫인코딩
2. 연령대  
순위변수(10년이 기준) - 20대: 2, 30대: 3, ..로 오디널인코딩
3. 거주지분류대코드  
명목변수 - 종류 매우 많음. 따라서 오디널인코딩
4. 제휴사  
명목변수 - B01, C01, C02, D01, D02, E01에 대해 오디널인코딩
5. 상품대분류명  
명목변수 - 종류 매우 많음. 따라서 오디널인코딩
6. 구매월  
명목변수 - 1월:1, 2월:2, ..로 오디널인코딩

In [15]:
# 성별 더미변수로 바꾸기 
df = pd.get_dummies(df, columns=['성별'])
df

,연령대,거주지분류대코드,제휴사,상품대분류명,구매시간,구매금액,구매수량,구매월,cluster,성별_남성,성별_여성
0,40대,Z10,A02,축산물,16,19560,1,08월,0,0,1
1,40대,Z10,A02,유제품,22,4980,1,12월,2,0,1
2,30대,Z11,A01,스포츠패션,16,124000,1,09월,1,0,1
3,40대,Z13,A02,완구,15,149900,1,12월,2,0,1
4,50대,Z10,A02,음료,20,5880,1,06월,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...
299995,50대,Z08,A02,과자,12,3180,1,09월,0,0,1
299996,20대,Z10,A03,채소,12,2490,1,10월,2,1,0
299997,40대,Z04,A02,대용식,13,6480,1,08월,0,0,1
299998,30대,Z17,A04,과자,7,1000,1,10월,0,0,1


In [16]:
# 연령대 숫자형으로 바꾸기
before = list(np.sort(df['연령대'].unique()))
print(before)
after = [i for i in range(len(before))]
print(after)
df['연령대'].replace(before, after, inplace=True)

['20대', '30대', '40대', '50대', '60대', '70대']
[0, 1, 2, 3, 4, 5]


In [17]:
# 거주지분류대코드 숫자형으로 바꾸기
before = list(np.sort(df['거주지분류대코드'].unique()))
print(before)
after = [i for i in range(len(before))]
print(after)
df['거주지분류대코드'].replace(before, after, inplace=True)

['Z01', 'Z02', 'Z03', 'Z04', 'Z05', 'Z06', 'Z07', 'Z08', 'Z09', 'Z10', 'Z11', 'Z12', 'Z13', 'Z14', 'Z15', 'Z16', 'Z17']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]


In [18]:
# 제휴사 숫자형으로 바꾸기
before = list(np.sort(df['제휴사'].unique()))
print(before)
after = [i for i in range(len(before))]
print(after)
df['제휴사'].replace(before, after, inplace=True)

['A01', 'A02', 'A03', 'A04', 'A05']
[0, 1, 2, 3, 4]


In [19]:
# 상품대분류명 숫자형으로 바꾸기
before = list(np.sort(df['상품대분류명'].unique()))
print(before)
after = [i for i in range(len(before))]
print(after)
df['상품대분류명'].replace(before, after, inplace=True)

['가구', '건강식품', '건강용품', '건해산물', '계절가전', '공구/안전용품', '과일', '과자', '구기/필드스포츠', '기타(비상품)', '남성의류', '냉동식품', '냉장/세탁가전', '냉장식품', '담배', '대용식', '모바일', '문구/사무용품', '병통조림', '상품권', '생활/렌탈서비스', '생활/주방가전', '서적/음반/악기', '세제/위생', '속옷/양말/홈웨어', '수산물', '스포츠패션', '시즌스포츠', '식기/조리기구', '아웃도어/레저', '양곡', '여성의류', '여행/레저서비스', '영상/음향가전', '완구', '원예/애완', '유아동의류', '유아식품', '유제품', '음료', '인테리어/조명', '자동차용품', '조리식품', '조미료', '주류', '주방잡화', '채소', '청소/세탁/욕실용품', '축산물', '출산/육아용품', '침구/수예', '커피/차', '컴퓨터', '테넌트/음식점', '패션잡화', '퍼스널케어', '헬스/피트니스', '화장품/뷰티케어']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57]


In [20]:
# 구매월 숫자형으로 바꾸기
before = list(np.sort(df['구매월'].unique()))
print(before)
after = [i for i in range(len(before))]
print(after)
df['구매월'].replace(before, after, inplace=True)

['01월', '02월', '03월', '04월', '05월', '06월', '07월', '08월', '09월', '10월', '11월', '12월']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [21]:
## 타입이 잘 바꼈는지 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 11 columns):
 #   Column    Non-Null Count   Dtype
---  ------    --------------   -----
 0   연령대       300000 non-null  int64
 1   거주지분류대코드  300000 non-null  int64
 2   제휴사       300000 non-null  int64
 3   상품대분류명    300000 non-null  int64
 4   구매시간      300000 non-null  int64
 5   구매금액      300000 non-null  int64
 6   구매수량      300000 non-null  int64
 7   구매월       300000 non-null  int64
 8   cluster   300000 non-null  int64
 9   성별_남성     300000 non-null  uint8
 10  성별_여성     300000 non-null  uint8
dtypes: int64(9), uint8(2)
memory usage: 21.2 MB


## X, Y 변수 나누기
train, test과 X, Y 이렇게 총 4가지 변수로 나눈다.

In [22]:
df.head()

,연령대,거주지분류대코드,제휴사,상품대분류명,구매시간,구매금액,구매수량,구매월,cluster,성별_남성,성별_여성
0,2,9,1,48,16,19560,1,7,0,0,1
1,2,9,1,38,22,4980,1,11,2,0,1
2,1,10,0,26,16,124000,1,8,1,0,1
3,2,12,1,34,15,149900,1,11,2,0,1
4,3,9,1,39,20,5880,1,5,1,0,1


In [24]:
# X 변수 설정
x_cols = ['연령대', '거주지분류대코드', '제휴사', '상품대분류명', '구매시간', '구매금액', '구매수량', '구매월', '성별_남성', '성별_여성']
X = df[x_cols].values
X

array([[ 2,  9,  1, ...,  7,  0,  1],
       [ 2,  9,  1, ..., 11,  0,  1],
       [ 1, 10,  0, ...,  8,  0,  1],
       ...,
       [ 2,  3,  1, ...,  7,  0,  1],
       [ 1, 16,  3, ...,  9,  0,  1],
       [ 1,  1,  1, ...,  9,  1,  0]], dtype=int64)

In [25]:
# Y 설정
y = df['cluster'].values
y

array([0, 2, 1, ..., 0, 0, 1], dtype=int64)

# Random Forest 모델 적합

In [33]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score 
from sklearn.metrics import precision_score 
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score 

## train, test 나누기

In [30]:
## train, test 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7) # 20%를 test로, random_state으로 랜덤한 값 고정
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(210000, 10) (90000, 10) (210000,) (90000,)


## Scaling

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## 최종 모델
군집의 범주가 4이기 때문에 이진분류가 아니라 다중분류모델이다.

In [ ]:
def model(algorithm, algorithm_name, feature_name, x_train, y_train, x_test, y_test, n_estimator, n_depth, n_split, n_leaf):):
    model = RandomForestClassifier(n_estimators=n_estimator, 
                                   max_depth=n_depth, 
                                   random_state=0)
    model.fit(X_train, Y_train)

    y_pred1 = model.predict(X_test)
    print("Train set 정확도: {:.3f}".format(model.score(X_train, y_train)))
    print("Test set 정확도: {:.3f}\n".format(model.score(X_test, y_test)))

    print(f"Accuracy: {accuracy_score(y_test, y_pred1):.3f}")  # 정확도
    print(f"Precision: {precision_score(y_test, y_pred1):.3f}")  # 정밀도
    print(f"Recall: {recall_score(y_test, y_pred1):.3f}")  # 재현율
    print(f"F1-score: {f1_score(y_test, y_pred1):.3f}")  # F1 스코어

In [55]:
## 1. sample - 100개, tree depth - 5

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0)
model.fit(X_train, y_train)
y_pred1 = model.predict(X_test)

print("Train set 정확도: {:.3f}".format(model.score(X_train, y_train)))
print("Test set 정확도: {:.3f}\n".format(model.score(X_test, y_test)))

print(f"Accuracy: {accuracy_score(y_test, y_pred1):.3f}") 
print(f"Precision: {precision_score(y_test, y_pred1, average='macro'):.3f}")  #micro보다 성능이 좋게 나옴
print(f"Recall: {recall_score(y_test, y_pred1, average='micro'):.3f}")
print(f"F1-score: {f1_score(y_test, y_pred1, average='micro'):.3f}")

Train set 정확도: 0.750
Test set 정확도: 0.750

Accuracy: 0.750
Precision: 0.772
Recall: 0.750
F1-score: 0.750


In [57]:
## 2. sample - 300개, tree depth - 20

model = RandomForestClassifier(n_estimators=300, max_depth=10, random_state=0)
model.fit(X_train, y_train)
y_pred2 = model.predict(X_test)

print("Train set 정확도: {:.3f}".format(model.score(X_train, y_train)))
print("Test set 정확도: {:.3f}\n".format(model.score(X_test, y_test)))

print(f"Accuracy: {accuracy_score(y_test, y_pred1):.3f}")  
print(f"Precision: {precision_score(y_test, y_pred1, average='macro'):.3f}")  
print(f"Recall: {recall_score(y_test, y_pred1, average='micro'):.3f}") 
print(f"F1-score: {f1_score(y_test, y_pred1, average='micro'):.3f}") 

Train set 정확도: 1.000
Test set 정확도: 0.990

Accuracy: 0.750
Precision: 0.772
Recall: 0.750
F1-score: 0.750


In [59]:
## 2. sample - 300개, tree depth - 20

model = RandomForestClassifier(n_estimators=150, max_depth=10, random_state=0)
model.fit(X_train, y_train)
y_pred2 = model.predict(X_test)

print("Train set 정확도: {:.3f}".format(model.score(X_train, y_train)))
print("Test set 정확도: {:.3f}\n".format(model.score(X_test, y_test)))

print(f"Accuracy: {accuracy_score(y_test, y_pred1):.3f}")  
print(f"Precision: {precision_score(y_test, y_pred1, average='macro'):.3f}")  
print(f"Recall: {recall_score(y_test, y_pred1, average='micro'):.3f}") 
print(f"F1-score: {f1_score(y_test, y_pred1, average='micro'):.3f}") 

Train set 정확도: 0.917
Test set 정확도: 0.915

Accuracy: 0.750
Precision: 0.772
Recall: 0.750
F1-score: 0.750


## Feature importance 확인

In [66]:
import seaborn as sns
import numpy as np
%matplotlib inline

In [ ]:
import seaborn as sns
import numpy as np
%matplotlib inline

X_train = pd.DataFrame(X_train, columns=['연령대', '거주지분류대코드', '제휴사', '상품대분류명', '구매시간', '구매금액', '구매수량', '구매월', '성별_남성', '성별_여성'])
feature_scores = pd.Series(model.feature_importances_, index=X_train.columns).sort_values(ascending=False)
feature_scores[:10]

# feature importance 추출 
print("Feature importances:\n{0}".format(np.round(model.feature_importances_, 3)))

# feature별 importance 매핑
for name, value in zip(cancer.feature_names , model.feature_importances_):
    print('{0} : {1:.3f}'.format(name, value))

# feature importance를 column 별로 시각화 하기 
sns.barplot(x=model.feature_importances_ , y=cancer.feature_names)